# Text Klassifikation - Sentiment Analyse

In diesem Notebook möchten wir uns mit der Klassifikation von Texten beschäftigen. Vereinfacht gesagt beschäftigt sich Sentiment Analysis damit, natürlichsprachliche Aussagen dahingehend zu bewerten, ob die subjektive Aussage des Sprechers positiv oder negativ wertend gemeint ist.

Zu diesem Zweck haben wir den Datensatz von _Sentiment140_, einem Projekt der Stanford University, ausgewählt. Er beinhaltet 16 Millionen Tweets, die aufgrund der enthaltenen Emoticons automatisch in positiv und negativ eingeteilt wurden.

---

Dieses Notebook gliedert sich in die folgenden Teile:

1. Datenanalyse und Preprocessing
2. Tokenisierung & Vokabular
3. Klassifikation
4. Erweiterung der Klassifikation

In [2]:
import tensorflow as tf
tf.__version__

'2.6.0'

## 0. Vorbereitung

Als erstes muss der Datensatz von http://help.sentiment140.com/for-students in diese Colab Virtual Machine geladen werden.

In [2]:
!pip install googledrivedownloader

In [3]:
#from google_drive_downloader import GoogleDriveDownloader as gdd

#gdd.download_file_from_google_drive(file_id='0B04GJPshIjmPRnZManQwWEdTZjg',
#                                    dest_path='./download/trainingandtestdata.zip',
#                                    unzip=False,
#                                    overwrite=True)

#DATA_DIR = 'download/sentiment140'

In [4]:
#!unzip download/trainingandtestdata.zip -d {DATA_DIR}
#%rm download/trainingandtestdata.zip
#%ls -la {DATA_DIR}

## 1. Datenbeschaffung und -Analyse

Im ersten Schritt sollen folgende Schritte durchgeführt werden.

1. Ladet den Datensatz in einen Pandas Dataframe. Welche Feature gibt es? Wie viele Samples gibt es?

2. Da wir uns nur für die Felder `polarity` und `text` interessieren, sollte der Frame mit den Daten folgendes Format haben : `id => (polarity, text)`. 

3. Wandelt die Polarity Werte in 1 (positiv) und 0 (negativ) um.

4. Fügt eine Spalte für die Anzahl an Wörtern im Text hinzu. Was ist die durschnittliche Tweetlänge

5. Analysiert den Datensatz mit beliebigen weiteren Pandas Boardmitteln.

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv(
    "training.1600000.processed.noemoticon.csv",
    encoding="ISO-8859-1",
    header=None,
    names=["polarity", "id", "date", "query", "user", "text"]
)

In [7]:
df = df[["polarity", "id", "text"]]

In [8]:
di = {0: 0, 4: 1}
df = df[df["polarity"] != 2].replace({"polarity": di})

In [9]:
df['word_count'] = df.apply(lambda row: len(row.text), axis=1)
print(f'Mean tweet length: {df.word_count.mean()}')

Mean tweet length: 74.09011125


In [10]:
df.groupby("polarity").describe()["word_count"]

,count,mean,std,min,25%,50%,75%,max
polarity,,,,,,,,
0,800000.0,74.301790,36.743260,6.0,44.0,70.0,104.0,359.0
1,800000.0,73.878433,36.135274,6.0,44.0,69.0,103.0,374.0


## 2. Tokenisierung

Um später ein Modell zur Sentiment Analyse trainieren zu können, sollen die Daten nun aufbereitet werden. Da die sinntragenden Elemente in den Tweets die Wörter sind, sollten die Tweets in Wörter aufgeteilt werden. Um genau zu sein, ist der Term 'Wörter' hier aus linguistischer Sicht etwas falsch, man spricht eigentlich von Tokens. Daher nennt man das Aufteilen von Text auch Tokenizing und die Funktion, die sowas kann, Tokenizer.

1. Der allereinfachste Tokenizer ist vermutlich die `split` Methode. Tokenisiert damit die eingelesen Tweets. Am Ende solltet ihr eine Liste `df['tokenized'] = [token_1,token_2, ...]` für jedes Sample erhalten.

In [11]:
df['tokenized'] = df.apply(lambda row: row.text.split(" "), axis=1)

Abgesehen von natürlichsprachlichen Wörtern sind in Tweets mindestens auch Hashtags, Mentions und Links enthalten. Überlegt euch, ob es Sinn ergibt, alle diese Bestandteile in den Daten in dieser Form zu behalten. Begründet kurz eure Entscheidungen.

Falls ihr euch entschlossen habt, nicht alle diese Bestandteile zu behalten, filtert dementsprechend eure Daten. Die Struktur der Daten sollte am Ende gleich bleiben: `df['cleaned'] = [token_1,...]`

In [12]:
import validators

def clean(token):
    if token.startswith("@"):
        return "<MENTION>"
    if token.startswith("#"):
        return "<HASHTAG>"
    if token == "&amp;":
        return "and"
    if token.startswith("http") and validators.url(token):
        return "<LINK>"
    return token

def filter_token(token):
    if token == "" or token == "-":
        return False
    return True

df['cleaned'] = df.apply(lambda row: [clean(token) for token in row.tokenized if filter_token(token)], axis=1)

Zählt die Tokens in eurem Datensatz. Benutzt dafür ein Dictionary oder andere Python Build-Ins. 

1. Wieviele unterschiedliche Wörter gibt es?
2. Gebt die 100 häufigsten Wörter sortiert aus. 

Was zieht ihr aus den beiden Analysen? Was müsst ihr zusätzlich noch filtern?

In [13]:
tokens = dict()
for index, row in df.iterrows():
    for token in row.cleaned:
        if not token in tokens:
            tokens[token] = 0
        tokens[token] += 1
        
tokens_sorted = sorted(tokens.items(), key=lambda item: item[1], reverse=True)
print(tokens_sorted[:100])


[('<MENTION>', 793948), ('to', 552962), ('I', 496608), ('the', 487500), ('a', 366212), ('and', 315391), ('my', 280025), ('i', 249975), ('is', 217692), ('you', 213871), ('for', 209800), ('in', 202294), ('of', 179554), ('it', 171810), ('on', 154365), ('have', 132249), ('so', 125154), ('me', 122509), ('that', 118684), ('with', 110843), ('be', 108069), ('but', 106271), ('at', 102196), ("I'm", 99559), ('was', 99140), ('just', 96284), ('not', 88110), ('this', 77809), ('get', 76733), ('like', 73302), ('are', 72568), ('<LINK>', 70538), ('up', 70007), ('all', 67901), ('out', 67030), ('go', 62969), ('your', 60854), ('good', 59775), ('day', 55748), ('do', 54628), ('from', 54182), ('got', 53870), ('now', 53591), ('going', 53236), ('love', 50051), ('no', 49621), ('about', 46708), ('work', 45913), ('will', 45898), ('<HASHTAG>', 44108), ('back', 44033), ('u', 43566), ("it's", 43422), ('some', 42745), ('am', 42724), ('can', 42506), ("don't", 42472), ('really', 42152), ('had', 41548), ('see', 41342), (

Eine Möglichkeit für komplexere Preprocessing-Methoden ist das Entfernen von Stoppwörtern. Hiefür nutzen wir [NLTK](https://www.nltk.org/).

Filtert eure gesäuberten Tokens auf Stopwörter.

In [14]:
import nltk
nltk.download("stopwords")
stopwords = nltk.corpus.stopwords.words("english")
df['cleaned'] = df.apply(lambda row: [token for token in row.cleaned if not token in stopwords], axis=1)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 3. Klassifikation

Wie Eingangs erwähnt, beschäftigt sich Sentiment Analysis damit, eine Äußerung automatisch dahingehend zu klassifizieren,
ob der Inhalt positiv oder negativ gemeint ist.
Es handelt sich also um eine binäre Klassifikation.

Für das Training des neuronalen Netzes orintiert sich der nachfolgende Teil an _Keras_ als Framework. Ihr könnt aber auch ein anderes Framework wie bspw. _Pytorch_ benutzen. 

Aktuell liegen unsere Daten zwar in tokenisierter und gesäuberter Form vor, wir müssen unsere Daten aber noch in Vektoren transformieren.

Für die erste Klassifikation encoden wir die Eingabe als **Bag-of-Words**, sodass jedes potentielle Wort einem Eingabeneuron einspricht. _Ein Beispiel_: Zwei Tweets "lorem ipsum" und "foo foo bar", die Vektoren hätten die Länge 4 und für den ersten Tweet wäre der Vektor [1,1,0,0], für den zweiten [0,0,2,1].

### 3.1 Vokabular

Befüllt das dictionary `word2idx` so, dass jedes Wort auf einen Index abgebildet wird und die Indizes streng monoton aufsteigend sind. Für das Beispiel oben wäre `word2idx = {"lorem": 0, "ipsum": 1, "foo": 3, "bar": 4}`

In [15]:
from tqdm import tqdm

word2idx = dict()
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    for token in row.cleaned:
        if token in word2idx: continue
        word2idx[token] = index

100%|██████████| 1600000/1600000 [01:34<00:00, 17005.28it/s]


Welche Länge werden die Vektoren haben?

In [16]:
VECTOR_LEN = len(word2idx)
print(VECTOR_LEN)

908070


Wir könnten mit `numpy` ein Array befüllen, das für jeden der 16 Millionen Tweets einen Vektor wie oben beschrieben enthält. 
Bevor ihr damit beginnen, überschlagt, wieviel Speicherplatz (im Hauptspeicher) ein solches Array belegen würde, wenn jeder Eintrag 32 bit hat. Reicht euer Hauptspeicher dafür aus?

In [17]:
MEMORY = 16000000 * 4 * VECTOR_LEN # 32 bit = 4byte
print(MEMORY / 1024 / 1024 / 1024) # GiB

54125.189781188965


### 3.2 Data Generator
Um das Problem mit dem zu kleinen Hauptspeicher zu umgehen, bietet Keras die Möglichkeit, anstatt auf einem kompletten Datensatz zu operieren, immer nur kleinere Häppchen abzuarbeiten. Dazu wird ein Python-Generator eingesetzt.
Vervollständigt die Funktion unten, so dass ein Generator entsteht. Die Parameter der Funktion sind:
 * x: tokenisierte und gesäuberte Tweets
 * y: korrespondierende Labels von x
 * w2i: das word2index dictionary
 * batch_size: Anzahl der vektorisierten Tweets, die pro Aufruf zurückgegeben werden sollen.
 
Die benutzen Tweets werden nacheinander aus `(x,y)` ausgewählt und kein Tweet darf mehrfach zurückgegeben werden.

In [29]:
from math import ceil
import random
import numpy as np

def data_generator(x, y, w2i, batch_size):
  num_batches = ceil(len(x) / batch_size)
  current_batch = 0

  while True:
    # Initialize with zeros
    batch_x = np.zeros((batch_size, len(w2i.keys())))
    batch_y = np.zeros((batch_size, 1))

    current_entry = 0
    for i in range(current_batch * batch_size, (current_batch + 1) * batch_size):
        for token in x[i]:
            batch_x[current_entry][w2i[token]] += 1
        batch_y[current_entry] = [y[i]]
        current_entry += 1
  
    current_batch += 1
    yield batch_x, batch_y

    # Reset counter
    if current_batch >= num_batches:
      current_batch = 0

In [30]:
gen = data_generator(df["cleaned"].to_numpy(), df["polarity"].to_numpy(), word2idx, 256) # TODO

Ihr könnt euren Generator wie folgt ausprobieren:

In [38]:
next(gen)

MemoryError: Unable to allocate 1.73 GiB for an array with shape (256, 908070) and data type float64

### 3.3 Neuronales Netz

Wir sind nun endlich soweit, unser neuronales Netz aufzubauen. Da unser Netz genau ein hidden Layer hat und auch sonst nicht sonderlich komplex ist, benutzen wir die _Sequential_-API von Keras.

In [4]:
from tensorflow.keras.models import Sequential
m = Sequential([
    tf.keras.Input(shape=(VECTOR_LEN,))
])

Fügt einen _Dense_-Layer dem Netz hinzu, als _hidden units_ könnt ihr 16 nehmen. Da dies auch der Eingabe-Layer ist, müsst ihr den Parameter `input_shape` definieren. (Siehe auch: https://keras.io/layers/core/)

In [6]:
m.add(tf.keras.layers.Dense(16, activation="relu")) # TODO

Als letzten Layer in unserem neuronalen Netz, fügt einen weiteren _Dense_-Layer hinzu. Dieser Layer dient auch als "Ausgabelayer" Überlegt euch die Anzahl der _hidden units_ (Hinweis: Wie lässt sich unser Machine-Learning-Problem kategorisieren?) Welche _Activation_-Funktion wählt ihr?

In [7]:
m.add(tf.keras.layers.Dense(2, activation="softmax"))

Kompiliert das neuronale Netz. Als `optimizer` könnt ihr `adam` benutzen. Wählt eine passende `loss`-Funktion aus. Begründet eure Entscheidung. (https://keras.io/models/model/#compile)

In [ ]:
m.compile(optimizer="adam", loss="categorical_crossentropy")

### 3.3. Training

Bevor ihr nun das neuronale Netz trainiert, teilt noch euren Datensatz in drei Teile auf. Einen Teil zum Trainieren, einen zum Evaluieren und einem zum Testen. Das Verhältnis der beiden Datensätze sollte 70%:20%:10% sein. Bevor ihr die Daten aufteilt, durchmischt sie mit der `shuffle`-Methode aus dem `random`-Modul. Außerdem solltet ihr die Datenmenge zunächst auf ca. 100000 begrenzen, damit das Training des neuronalen Netzes nicht ewig dauert.

In [ ]:
df_shuffeled = df.sample(frac=1).reset_index(drop=True).head(100000)
df_train = df_shuffeled.loc[0:len(df_shuffeled) * 0.7]
df_val = df_shuffeled.loc[len(df_shuffeled) * 0.7:len(df_shuffeled) * 0.9]
df_test = df_shuffeled.loc[len(df_shuffeled) * 0.9:len(df_shuffeled)]

assert((len(df_train) + len(df_val) + len(df_test)) == len(df_shuffeled))

Wir sind nun soweit das neuronale Netz zu trainieren. Da wir den oben entwickelten Generator einsetzen wollen, verwenden wir dazu die `fit_generator`-Methode. Als `batch_size` könnt ihr 100 nehmen, für den `epochs`-Parameter 10. Was wählt ihr als `steps_per_epoch`-Parameter? Wie übergebt ihr die Validierungsdaten? (https://keras.io/models/model/#fit)

In [ ]:
batch_size = 100

m.fit(
    data_generator(df_train.cleaned.to_numpy(), df_train.polarity.to_numpy(), word2idx, batch_size),
    epochs=10,
    steps_per_epoch=len(df_train) / batch_size,
    validation_data=data_generator(df_val.cleaned.to_numpy(), df_val.polarity.to_numpy(), word2idx, batch_size)
)

Während das Netz trainiert wird, könnt ihr euch Gedanken zur Evaluierung machen:
   * Definiert die üblichen Fehlerklassen (wahr positiv, falsch positiv, wahr negativ, falsch negativ)
   * Eine häufig benutzte Evaluationsmetrik ist die _Accuracy_. Beschreibt diese Metrik und schreibt die Formel zur Berechnung auf.
   * Warum könnte die _Accuracy_ eine schlechte Metrik sein?
   * Zur Evaluation von binären Klassifikationsproblemen wird in der Literatur gerne _Precision_ und _Recall_ verwendet. Wie sind die beiden Evaluationsmaße definiert? Beschreibt diese Metriken mit eigenen Worten. Schreibt auch die Formeln zur Berechnung auf.
   * Warum könnten _Precision_ und _Recall_ bessere Metriken sein als _Accuracy_?

11. Inzwischen sollte das Netz fertig trainiert sein. Speichert es ab!

In [ ]:
m.save('my_net.h5')

### 3.4. Evaluation

Evaluiert euer Netz mit dem Test-Datensatz, den ihr oben beseite gelegt haben. Benutzt dafür die `predict`-Methode des Models. Berechnet dafür _Precision_, _Recall_, _Accuracy_ und _Confusion Matrix_. 

Interpretiert kurz eure Ergebnisse. 

In [ ]:
# TODO

## _(Optional)_ 4. Erweiterung der Klassifikation
Im nächsten Schritt wollen wir das Netz auf dem vollen Datensatz trainieren und die Komplexität etwas erhöhen.

1. Trainiert euer Netz auf dem großen Datensatz. Was sind die Erkentnisse?

In [ ]:
# TODO

2. Verändert die Parameter Ihres Netzes (z.B Anzahl _hidden units_, Anzahl _hidden layers_) und trainiert das Netz erneut (auf dem kleinen Datensatz). Was stellt ihr fest?

In [ ]:
# TODO